In [1]:
%load_ext autoreload

In [2]:
from _context import src
#from src.models import TransformerClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchtext import data, datasets, vocab

import numpy as np
import random, tqdm, sys, math, gzip


## Dataset

In [6]:
text = data.Field(lower=True, include_lengths=True, batch_first=True)
lalbel = data.Field(sequential=False)

/home/satellogic/anaconda3/envs/transformers_from_scratch/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
